# Identifying Adverbs with Strongest Negative Environment Associations

In [1]:
from pathlib import Path

import pandas as pd
from pprint import pprint

from source.utils import PKL_SUFF, SAMPLE_ADV, print_md_table
from source.utils.associate import AM_DF_DIR, TOP_AM_DIR, adjust_assoc_columns
from source.utils.general import print_iter, snake_to_camel, timestamp_today

SET_FLOOR = 2000
MIR_FLOOR = 200
K = 5

/home/arh234/anaconda3/envs/dev-sanpi/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Set columns and diplay settings

In [2]:
FOCUS = ['f', 'E11', 'unexpected_f',
         'am_p1_given2', 'conservative_log_ratio',
         'am_log_likelihood', 't_score',
         'mutual_information', 'am_odds_ratio_disc',
         'N', 'f1', 'f2', 'l1', 'l2']
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.max_columns', 12)
pd.set_option('display.width', 90)
pd.set_option("display.precision", 2)
pd.set_option("styler.format.precision", 2)
pd.set_option("styler.format.thousands", ",")
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("styler.render.repr", "html")

## Set paths and load adverb association tables

In [3]:
POLAR_DIR = AM_DF_DIR.joinpath('polar')

polar_adv_dirs = []
# results/assoc_df/polar/RBdirect/adv/extra/polarized-adv_35f-7c_min5000x_extra.pkl.gz
adv_am_paths = {
    p.name: tuple(
        p.joinpath('adv/extra').glob(
            f'*35f-7c_min{SET_FLOOR if p.name == "RBdirect" else MIR_FLOOR}x*{PKL_SUFF}')
    )[0]
    for p in POLAR_DIR.iterdir()}

setdiff_adv = pd.read_pickle(adv_am_paths['RBdirect'])
mirror_adv = pd.read_pickle(adv_am_paths['NEGmirror'])

In [4]:
adjust_assoc_columns(setdiff_adv.sample(K).sort_values('conservative_log_ratio', ascending=False)[FOCUS])

,f,exp_f,unexp_f,dP1,LRC,G2,...,odds_r_disc,N,f1,f2,l1,l2
key,,,,,,,,,,,,,
COM~darkly,10447,"10,086.13",360.87,0.03,2.38,577.25,...,1.11,86330752,83102035,10478,COMPLEMENT,darkly
COM~dangerously,31481,"30,505.78",975.22,0.03,2.03,"1,254.76",...,0.76,86330752,83102035,31691,COMPLEMENT,dangerously
COM~ironically,2128,"2,065.74",62.26,0.03,0.45,72.56,...,0.65,86330752,83102035,2146,COMPLEMENT,ironically
COM~super,308753,"306,681.67","2,071.33",0.01,0.21,398.31,...,0.09,86330752,83102035,318597,COMPLEMENT,super
NEG~extremely,5543,"37,074.93","-31,531.93",-0.03,-2.69,"-43,348.64",...,-0.84,86330752,3226213,992094,NEGATED,extremely


In [5]:
adjust_assoc_columns(mirror_adv.sample(K).sort_values('conservative_log_ratio', ascending=False)[FOCUS])

,f,exp_f,unexp_f,dP1,LRC,G2,...,odds_r_disc,N,f1,f2,l1,l2
key,,,,,,,,,,,,,
POS~intensely,788,690.25,97.75,0.12,1.13,139.77,...,0.84,2032082,1738105,807,POSMIR,intensely
POS~unusually,630,555.97,74.03,0.11,0.77,95.62,...,0.72,2032082,1738105,650,POSMIR,unusually
POS~clinically,274,267.72,6.28,0.02,0.00,1.06,...,0.07,2032082,1738105,313,POSMIR,clinically
POS~plainly,263,231.79,31.21,0.12,0.00,41.01,...,0.72,2032082,1738105,271,POSMIR,plainly
POS~marginally,289,257.45,31.55,0.10,0.00,35.88,...,0.59,2032082,1738105,301,POSMIR,marginally


## Calculate "Most Negative" Adverbs for each Polarity Approximation

In [6]:
def get_top_vals(df: pd.DataFrame,
                 index_like: str = 'NEG',
                 metric_filter: str | list = 'conservative_log_ratio',
                 k: int = 10,
                 val_col: str = None,
                 ignore_neg_adv: bool = True):
    env_df = df.copy().loc[df.conservative_log_ratio >=
                           1].filter(like=index_like, axis=0)
    if ignore_neg_adv:
        env_df = env_df.loc[~df.l2.isin(
            ("n't", 'not', 'barely', 'never', 'no', 'none')), :]
    if isinstance(metric_filter, str):
        metric_filter = [metric_filter]

    top = pd.concat([env_df.nlargest(k, m) for m in metric_filter]
                    ).drop_duplicates(keep='first')

    if val_col:
        top = top[[val_col] + metric_filter]

    return top.sort_values(metric_filter[0], ascending=False)


[setdiff_top15, mirror_top15] = [
    get_top_vals(
        adv_df, k=15,
        metric_filter=['am_p1_given2', 'conservative_log_ratio'])
    for adv_df in (setdiff_adv, mirror_adv)
]
adjust_assoc_columns(setdiff_top15.filter(items=FOCUS).reset_index())

,key,f,exp_f,unexp_f,dP1,LRC,...,odds_r_disc,N,f1,f2,l1,l2
0,NEG~necessarily,42708,"2,118.68","40,589.32",0.72,6.23,...,1.90,86330752,3226213,56694,NEGATED,necessarily
1,NEG~exactly,43635,"2,301.98","41,333.02",0.67,5.90,...,1.80,86330752,3226213,61599,NEGATED,exactly
2,NEG~that,165411,"9,357.24","156,053.76",0.63,5.62,...,1.72,86330752,3226213,250392,NEGATED,that
3,NEG~immediately,57319,"3,855.76","53,463.24",0.52,4.96,...,1.52,86330752,3226213,103177,NEGATED,immediately
4,NEG~yet,52546,"3,800.83","48,745.17",0.48,4.74,...,1.45,86330752,3226213,101707,NEGATED,yet
5,NEG~terribly,18054,"2,622.43","15,431.57",0.22,3.09,...,0.95,86330752,3226213,70174,NEGATED,terribly
6,NEG~remotely,5679,829.40,"4,849.60",0.22,3.03,...,0.95,86330752,3226213,22194,NEGATED,remotely
7,NEG~only,114070,"17,346.13","96,723.87",0.21,3.04,...,0.94,86330752,3226213,464168,NEGATED,only
8,NEG~altogether,4575,771.17,"3,803.82",0.18,2.75,...,0.87,86330752,3226213,20636,NEGATED,altogether
9,NEG~entirely,63708,"11,354.35","52,353.65",0.17,2.74,...,0.84,86330752,3226213,303833,NEGATED,entirely


In [7]:
adjust_assoc_columns(mirror_top15.filter(items=FOCUS).reset_index())

,key,f,exp_f,unexp_f,dP1,LRC,...,odds_r_disc,N,f1,f2,l1,l2
0,NEG~before,290,42.53,247.47,0.84,5.11,...,2.58,2032082,293963,294,NEGMIR,before
1,NEG~ever,4718,749.20,"3,968.80",0.77,5.57,...,1.79,2032082,293963,5179,NEGMIR,ever
2,NEG~exactly,813,161.15,651.85,0.59,3.51,...,1.20,2032082,293963,1114,NEGMIR,exactly
3,NEG~any,1082,219.02,862.98,0.57,3.48,...,1.17,2032082,293963,1514,NEGMIR,any
4,NEG~remotely,1846,393.04,"1,452.96",0.54,3.35,...,1.10,2032082,293963,2717,NEGMIR,remotely
5,NEG~particularly,9278,"2,163.26","7,114.74",0.48,3.15,...,1.00,2032082,293963,14954,NEGMIR,particularly
6,NEG~that,4338,"1,080.91","3,257.09",0.44,2.86,...,0.92,2032082,293963,7472,NEGMIR,that
7,NEG~necessarily,971,243.18,727.82,0.43,2.66,...,0.91,2032082,293963,1681,NEGMIR,necessarily
8,NEG~inherently,2872,817.19,"2,054.81",0.36,2.42,...,0.79,2032082,293963,5649,NEGMIR,inherently
9,NEG~overtly,392,129.91,262.09,0.29,1.71,...,0.66,2032082,293963,898,NEGMIR,overtly


In [8]:
def load_setdiff_backup(lower_floor: int = 100):
    backup_set_df = pd.read_pickle(tuple(adv_am_paths['RBdirect'].parent.glob(
        f'*35f-7c_min{lower_floor}x*{PKL_SUFF}'))[0])

    neg_set_backup = backup_set_df.filter(like='NEG', axis=0).filter(
        items=FOCUS).reset_index().set_index('l2')
    neg_set_backup.columns = pd.Series(
        adjust_assoc_columns(neg_set_backup.columns)) + '_SET'
    # print(neg_set_backup.head())
    return neg_set_backup

In [9]:
def combine_top(df_1: pd.DataFrame,
                name_1: str,
                df_2: pd.DataFrame,
                name_2: str,
                env_filter: str = 'NEG',
                filter_items: list = FOCUS,
                k: int = 10) -> pd.DataFrame:

    top_dfs = [get_top_vals(adv_df, index_like=env_filter, k=k,
                            metric_filter=['am_p1_given2',
                                           'conservative_log_ratio']
                            )
               for adv_df in [df_1, df_2]]
    for i, name in enumerate([name_1, name_2]):

        print_iter(
            top_dfs[i].l2.to_list(), bullet='1.',
            header=f'{name}: union of top {k} adverbs ranked by deltaP(1|2) and LRC')
    # print_iter(top_dfs[1].l2.to_list())

    top_adv = pd.concat((top_dfs[0].l2, top_dfs[1].l2)).drop_duplicates()
    # top_adv = top_dfs[0].l2.to_list()
    # for adv in top_dfs[1].l2:
    #     if adv not in top_adv:
    #         top_adv.append(adv)
    print_iter(
        top_adv, bullet='1.',
        header=f'Union of top adverbs for {name_1} and {name_2}. (Novel {name_2} adverbs listed last)')

    df_1, df_2 = [d.filter(items=filter_items)
                  .filter(like=env_filter, axis=0)
                  .reset_index().set_index('l2')
                  for d in [df_1, df_2]]
    df_1 = adjust_assoc_columns(df_1)
    df_2 = adjust_assoc_columns(df_2)

    both = pd.DataFrame(index=top_adv)
    name_1, name_2 = [f"_{n.strip('_')}" for n in [name_1, name_2]]
    both = both.join(df_1).join(df_2, lsuffix=name_1,
                                rsuffix=name_2).sort_index(axis=1)
    # ! Empty cells need to be filled _before_ calculating mean
    if any(both.f_SET.isna()):
        neg_set_backup = load_setdiff_backup()
        undefined = both.index[both.f_SET.isna()].to_list()
        both.loc[undefined,
                 neg_set_backup.columns] = neg_set_backup.loc[undefined, :]
        print_md_table(both.loc[undefined, :].select_dtypes(include='number').round(2), transpose=True, n_dec=2,
                       title=f'Retrieved values for Adverbs missing from loaded `set_diff` data (i.e. negated SET_DIFF tokens < {SET_FLOOR:,})')

    # for metric in ['LRC', 'dP1', 'G2', 'MI']:
    for metric in both.select_dtypes(include='number').columns.to_series().str.replace(r'_(MIR|SET)$', '', regex=True).unique():
        # print(metric)
        # print(both.filter(regex=f"^{metric}"))
        both[f'mean_{snake_to_camel(metric)}'] = both.filter(
            regex=f"^{metric}").agg('mean', axis='columns')

    return both

## Compile top NEG~adverb associations across both approximation methods

In [10]:
C = combine_top(setdiff_adv.copy(), 'SET',
                mirror_adv.copy(), 'MIR', k=K)


SET: union of top 5 adverbs ranked by deltaP(1|2) and LRC
1. necessarily
1. exactly
1. that
1. immediately
1. yet

MIR: union of top 5 adverbs ranked by deltaP(1|2) and LRC
1. before
1. ever
1. exactly
1. any
1. remotely

Union of top adverbs for SET and MIR. (Novel MIR adverbs listed last)
1. necessarily
1. exactly
1. that
1. immediately
1. yet
1. before
1. ever
1. any
1. remotely

Retrieved values for Adverbs missing from loaded `set_diff` data (i.e. negated SET_DIFF tokens < 2,000)

|                 |        before |
|:----------------|--------------:|
| G2_MIR          |      1,080.52 |
| G2_SET          |      1,062.13 |
| LRC_MIR         |          5.11 |
| LRC_SET         |          3.65 |
| MI_MIR          |          0.83 |
| MI_SET          |          1.05 |
| N_MIR           |  2,032,082.00 |
| N_SET           | 86,330,760.00 |
| dP1_MIR         |          0.84 |
| dP1_SET         |          0.38 |
| exp_f_MIR       |         42.53 |
| exp_f_SET       |         27.95 |
| f1

SET: union of top 5 adverbs ranked by $\Delta P(\texttt{env}|\texttt{adv})$ and LRC
1. necessarily
1. exactly
1. that
1. immediately
1. yet

MIR: union of top 5 adverbs ranked by $\Delta P(\texttt{env}|\texttt{adv})$ and LRC
1. before
1. ever
1. exactly
1. any
1. remotely

Union of top adverbs for SET and MIR. (Novel MIR adverbs listed last)
1. necessarily
1. exactly
1. that
1. immediately
1. yet
1. before
1. ever
1. any
1. remotely

In [11]:
print(C.columns.tolist())


['G2_MIR', 'G2_SET', 'LRC_MIR', 'LRC_SET', 'MI_MIR', 'MI_SET', 'N_MIR', 'N_SET', 'dP1_MIR', 'dP1_SET', 'exp_f_MIR', 'exp_f_SET', 'f1_MIR', 'f1_SET', 'f2_MIR', 'f2_SET', 'f_MIR', 'f_SET', 'key_MIR', 'key_SET', 'l1_MIR', 'l1_SET', 'odds_r_disc_MIR', 'odds_r_disc_SET', 't_MIR', 't_SET', 'unexp_f_MIR', 'unexp_f_SET', 'mean_G2', 'mean_LRC', 'mean_MI', 'mean_N', 'mean_dP1', 'mean_expF', 'mean_f1', 'mean_f2', 'mean_f', 'mean_oddsRDisc', 'mean_t', 'mean_unexpF']


In [12]:
main_cols_ordered = pd.concat((*[C.filter(like=m).columns.to_series() for m in ('LRC', 'P1', 'G2')],
                               *[C.filter(regex=f'^{f}_').columns.to_series() for f in ['f', 'f1', 'f2'] ]) 
                              ).to_list()
print(main_cols_ordered)

['LRC_MIR', 'LRC_SET', 'mean_LRC', 'dP1_MIR', 'dP1_SET', 'mean_dP1', 'G2_MIR', 'G2_SET', 'mean_G2', 'f_MIR', 'f_SET', 'f1_MIR', 'f1_SET', 'f2_MIR', 'f2_SET']


In [13]:
C.index.name = 'adv'
C=C.sort_values('mean_LRC', ascending=False)
pd.set_option('display.max_columns', 16)
C[main_cols_ordered]

,LRC_MIR,LRC_SET,mean_LRC,dP1_MIR,dP1_SET,mean_dP1,G2_MIR,G2_SET,mean_G2,f_MIR,f_SET,f1_MIR,f1_SET,f2_MIR,f2_SET
adv,,,,,,,,,,,,,,,
exactly,3.51,5.90,4.71,0.59,0.67,0.63,"1,939.47","214,404.20","108,171.83",813,"43,635.00",293963,"3,226,213.00",1114,"61,599.00"
necessarily,2.66,6.23,4.44,0.43,0.72,0.57,"1,688.91","219,003.46","110,346.18",971,"42,708.00",293963,"3,226,213.00",1681,"56,694.00"
before,5.11,3.65,4.38,0.84,0.38,0.61,"1,080.52","1,062.13","1,071.32",290,311.00,293963,"3,226,213.00",294,748.00
that,2.86,5.62,4.24,0.44,0.63,0.53,"7,632.21","781,016.11","394,324.16",4338,"165,411.00",293963,"3,226,213.00",7472,"250,392.00"
remotely,3.35,3.03,3.19,0.54,0.22,0.38,"4,009.84","13,354.33","8,682.08",1846,"5,679.00",293963,"3,226,213.00",2717,"22,194.00"
yet,1.18,4.74,2.96,0.21,0.48,0.34,242.23,"209,055.78","104,649.01",320,"52,546.00",293963,"3,226,213.00",909,"101,707.00"
ever,5.57,0.28,2.92,0.77,0.01,0.39,"15,340.34",353.58,"7,846.96",4718,"5,967.00",293963,"3,226,213.00",5179,"124,592.00"
immediately,0.79,4.96,2.88,0.14,0.52,0.33,181.20,"239,462.58","119,821.89",407,"57,319.00",293963,"3,226,213.00",1442,"103,177.00"
any,3.48,2.28,2.88,0.57,0.13,0.35,"2,511.26","23,683.00","13,097.13",1082,"15,492.00",293963,"3,226,213.00",1514,"94,152.00"


Save full adverb selection as `.csv`

In [14]:
C.to_csv(TOP_AM_DIR / f'Top{K}_NEG-ADV_combined.35f-7c_{timestamp_today()}.csv')

Save `all-columns`, `means`, and `MAIN` as markdown formatted tables

In [15]:
C.to_markdown(floatfmt=',.2f', intfmt=',', buf=TOP_AM_DIR / f'Top{K}_NEG-ADV_combined_all-columns.35f-7c_{timestamp_today()}.md')
C.filter(like='mean_').to_markdown(floatfmt=',.2f', intfmt=',', buf=TOP_AM_DIR / f'Top{K}_NEG-ADV_combined_means.35f-7c_{timestamp_today()}.md')
C[main_cols_ordered].to_markdown(floatfmt=',.2f', intfmt=',', buf=TOP_AM_DIR / f'Top{K}_NEG-ADV_combined_MAIN.35f-7c_{timestamp_today()}.md')

## Collect bigrams corresponding to top adverbs

In [16]:
# results/assoc_df/polar/RBdirect/bigram/polarized-bigram_35f-7c_min1000x.pkl.gz
bigram_floor = 200
bigram_dfs = {d.name:
              pd.read_pickle(
                  tuple(d.joinpath('bigram/extra')
                        .glob(f'*35f-7c*min{bigram_floor//2 if d.name == "NEGmirror" else bigram_floor}x*.pkl.gz')
                        )[0])
              for d in POLAR_DIR.iterdir()}

In [17]:
def show_adv_bigrams(sample_size, C, bigram_dfs, column_list: list = None) -> dict:

    print('# Top bigrams corresponding to top adverbs\n')
    print(timestamp_today())
    patterns = list(bigram_dfs.keys())
    top_adverbs = C.mean_LRC.nlargest(sample_size).index
    bigram_samples = dict.fromkeys(top_adverbs)
    bigrams = []
    adj = []
    for adv in top_adverbs:
        print(f'\n## _{adv}_\n')
        adv_top = None
        bigram_samples[adv] = dict.fromkeys(patterns + ['both', 'adj'])
        adj_for_adv = []
        for pat, bdf in bigram_dfs.items():
            bdf = bdf[FOCUS+['adv', 'adj', 'adj_total']]
            bdf.columns = adjust_assoc_columns(bdf.columns)
            bdf = bdf.loc[bdf.LRC >= 1, :]

            adv_pat_bigrams = bdf.filter(
                like=adv, axis=0).nlargest(sample_size, 'LRC')
            # print(adv_top_bigrams)
            if adv_pat_bigrams.empty:
                print(f'No bigrams found in loaded `{pat}` AM table.')
            else:
                column_list = column_list or bdf.columns
                print_md_table(adv_pat_bigrams[column_list], n_dec=2,
                               title=f'### Top {sample_size} `{pat}` "{adv}_*" bigrams (sorted by `LRC`; `LRC > 1`)')

            adj_for_adv.extend(adv_pat_bigrams.adj.drop_duplicates().to_list())

            bigram_samples[adv][pat] = adv_pat_bigrams

            adv_top = adv_pat_bigrams if adv_top is None else pd.concat(
                [adv_top, adv_pat_bigrams])

        bigram_samples[adv]['adj'] = set(adj_for_adv)
        bigrams.extend(adv_top.l2.drop_duplicates().to_list())
        adj.extend(adj_for_adv)
        bigram_samples[adv]['both'] = adv_top
    bigram_samples['bigrams'] = set(bigrams)
    bigram_samples['adj'] = set(adj)
    return bigram_samples


samples_dict = show_adv_bigrams(
    K, C, bigram_dfs,
    column_list=[
        'adj',
        *pd.Series(main_cols_ordered).str.replace(
            r'mean_|_SET|_MIR', '', regex=True)
        .drop_duplicates().to_list(),
        't', 'MI'
    ]
)

# Top bigrams corresponding to top adverbs

2024-05-15

## _exactly_


### Top 5 `RBdirect` "exactly_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key               | adj   |   LRC |   dP1 |        G2 |     f |        f1 |    f2 |     t |   MI |
|:------------------|:------|------:|------:|----------:|------:|----------:|------:|------:|-----:|
| NEG~exactly_sure  | sure  |  8.63 |  0.92 | 54,750.58 | 8,860 | 3,226,213 | 9,301 | 90.43 | 1.41 |
| NEG~exactly_new   | new   |  8.54 |  0.93 |  8,697.93 | 1,378 | 3,226,213 | 1,418 | 35.69 | 1.42 |
| NEG~exactly_easy  | easy  |  8.37 |  0.93 |  6,747.64 | 1,069 | 3,226,213 | 1,100 | 31.44 | 1.42 |
| NEG~exactly_clear | clear |  8.30 |  0.92 | 10,937.16 | 1,759 | 3,226,213 | 1,835 | 40.31 | 1.41 |
| NEG~exactly_cheap | cheap |  8.28 |  0.95 |  4,443.27 |   693 | 3,226,213 |   704 | 25.33 | 1.42 |


### Top 5 `NEGmirror` "exactly_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key              | adj   |   LRC |   dP1 |     G2 |   f |      f1 |   f2 | 

# Top bigrams corresponding to top adverbs

2024-05-15

## _exactly_


### Top 5 `RBdirect` "exactly_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key               | adj   |   LRC |   dP1 |        G2 |     f |        f1 |    f2 |     t |   MI |
|:------------------|:------|------:|------:|----------:|------:|----------:|------:|------:|-----:|
| NEG~exactly_sure  | sure  |  8.63 |  0.92 | 54,750.58 | 8,860 | 3,226,213 | 9,301 | 90.43 | 1.41 |
| NEG~exactly_new   | new   |  8.54 |  0.93 |  8,697.93 | 1,378 | 3,226,213 | 1,418 | 35.69 | 1.42 |
| NEG~exactly_easy  | easy  |  8.37 |  0.93 |  6,747.64 | 1,069 | 3,226,213 | 1,100 | 31.44 | 1.42 |
| NEG~exactly_clear | clear |  8.30 |  0.92 | 10,937.16 | 1,759 | 3,226,213 | 1,835 | 40.31 | 1.41 |
| NEG~exactly_cheap | cheap |  8.28 |  0.95 |  4,443.27 |   693 | 3,226,213 |   704 | 25.33 | 1.42 |


### Top 5 `NEGmirror` "exactly_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key              | adj   |   LRC |   dP1 |     G2 |   f |      f1 |   f2 |     t |   MI |
|:-----------------|:------|------:|------:|-------:|----:|--------:|-----:|------:|-----:|
| NEG~exactly_sure | sure  |  2.09 |  0.85 | 560.65 | 148 | 293,963 |  149 | 10.39 | 0.84 |


## _necessarily_


### Top 5 `RBdirect` "necessarily_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key                            | adj            |   LRC |   dP1 |        G2 |     f |        f1 |    f2 |     t |   MI |
|:-------------------------------|:---------------|------:|------:|----------:|------:|----------:|------:|------:|-----:|
| NEG~necessarily_indicative     | indicative     |  8.37 |  0.93 |  8,811.69 | 1,406 | 3,226,213 | 1,456 | 36.05 | 1.41 |
| NEG~necessarily_representative | representative |  7.31 |  0.91 |  3,044.27 |   496 | 3,226,213 |   524 | 21.39 | 1.40 |
| NEG~necessarily_easy           | easy           |  7.26 |  0.88 |  5,448.34 |   914 | 3,226,213 |   996 | 29.00 | 1.39 |
| NEG~necessarily_surprising     | surprising     |  7.22 |  0.93 |  2,150.86 |   343 | 3,226,213 |   355 | 17.80 | 1.41 |
| NEG~necessarily_true           | true           |  6.89 |  0.82 | 18,199.76 | 3,238 | 3,226,213 | 3,786 | 54.42 | 1.36 |


### Top 5 `NEGmirror` "necessarily_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key                   | adj   |   LRC |   dP1 |     G2 |   f |      f1 |   f2 |     t |   MI |
|:----------------------|:------|------:|------:|-------:|----:|--------:|-----:|------:|-----:|
| NEG~necessarily_wrong | wrong |  4.19 |  0.77 | 693.55 | 213 | 293,963 |  233 | 12.29 | 0.80 |


## _before_

No bigrams found in loaded `RBdirect` AM table.

### Top 5 `NEGmirror` "before_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key                  | adj       |   LRC |   dP1 |     G2 |   f |      f1 |   f2 |     t |   MI |
|:---------------------|:----------|------:|------:|-------:|----:|--------:|-----:|------:|-----:|
| NEG~before_available | available |  3.99 |  0.84 | 654.92 | 177 | 293,963 |  180 | 11.35 | 0.83 |


## _that_


### Top 5 `RBdirect` "that_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key                  | adj         |   LRC |   dP1 |        G2 |     f |        f1 |     f2 |     t |   MI |
|:---------------------|:------------|------:|------:|----------:|------:|----------:|-------:|------:|-----:|
| NEG~that_uncommon    | uncommon    |  8.39 |  0.94 |  5,136.91 |   804 | 3,226,213 |    819 | 27.28 | 1.42 |
| NEG~that_surprising  | surprising  |  8.14 |  0.92 |  7,115.30 | 1,141 | 3,226,213 |  1,187 | 32.47 | 1.41 |
| NEG~that_common      | common      |  8.12 |  0.92 |  7,564.08 | 1,216 | 3,226,213 |  1,268 | 33.51 | 1.41 |
| NEG~that_hard        | hard        |  7.96 |  0.88 | 59,642.82 | 9,966 | 3,226,213 | 10,818 | 95.78 | 1.39 |
| NEG~that_complicated | complicated |  7.95 |  0.91 |  7,450.89 | 1,208 | 3,226,213 |  1,270 | 33.39 | 1.41 |


### Top 5 `NEGmirror` "that_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key             | adj    |   LRC |   dP1 |       G2 |   f |      f1 |   f2 |     t |   MI |
|:----------------|:-------|------:|------:|---------:|----:|--------:|-----:|------:|-----:|
| NEG~that_simple | simple |  4.48 |  0.74 | 1,483.32 | 478 | 293,963 |  540 | 18.29 | 0.79 |
| NEG~that_easy   | easy   |  3.91 |  0.68 | 1,278.04 | 458 | 293,963 |  558 | 17.63 | 0.75 |
| NEG~that_big    | big    |  2.99 |  0.66 |   308.12 | 113 | 293,963 |  140 |  8.72 | 0.75 |
| NEG~that_good   | good   |  2.65 |  0.47 |   848.28 | 449 | 293,963 |  732 | 16.19 | 0.63 |
| NEG~that_great  | great  |  1.93 |  0.36 |   406.36 | 288 | 293,963 |  575 | 12.07 | 0.54 |


## _remotely_


### Top 5 `RBdirect` "remotely_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key                     | adj        |   LRC |   dP1 |       G2 |   f |        f1 |    f2 |     t |   MI |
|:------------------------|:-----------|------:|------:|---------:|----:|----------:|------:|------:|-----:|
| NEG~remotely_true       | true       |  4.46 |  0.56 | 1,089.49 | 250 | 3,226,213 |   420 | 14.82 | 1.20 |
| NEG~remotely_close      | close      |  2.92 |  0.23 | 1,722.76 | 696 | 3,226,213 | 2,558 | 22.76 | 0.86 |
| NEG~remotely_interested | interested |  2.72 |  0.23 |   808.74 | 333 | 3,226,213 | 1,252 | 15.68 | 0.85 |


### Top 5 `NEGmirror` "remotely_*" bigrams (sorted by `LRC`; `LRC > 1`)

| key                | adj   |   LRC |   dP1 |     G2 |   f |      f1 |   f2 |     t |   MI |
|:-------------------|:------|------:|------:|-------:|----:|--------:|-----:|------:|-----:|
| NEG~remotely_close | close |  3.02 |  0.59 | 524.61 | 219 | 293,963 |  299 | 11.88 | 0.70 |

In [18]:
pprint({key: adv_dict['adj'] for key, adv_dict in samples_dict.items() if key not in {'bigrams', 'adj'}})

{'before': {'available'},
 'exactly': {'easy', 'clear', 'cheap', 'new', 'sure'},
 'necessarily': {'easy',
                 'indicative',
                 'representative',
                 'surprising',
                 'true',
                 'wrong'},
 'remotely': {'interested', 'close', 'true'},
 'that': {'big',
          'common',
          'complicated',
          'easy',
          'good',
          'great',
          'hard',
          'simple',
          'surprising',
          'uncommon'}}


In [19]:
for key, info in samples_dict.items():
    if key in ('bigrams', 'adj'):
        key = f'ALL {key.replace("adj", "adjectives")}'
    formatted_iter = [
        f'_{a.replace("_", " ")}_' for a
        in (info['adj'] if isinstance(info, dict)
            else info)]
    print_iter(formatted_iter,
               header=f'+ _{key}_ ({len(formatted_iter)} unique)',
               bullet='+', indent=2)


+ _exactly_ (5 unique)
  + _easy_
  + _clear_
  + _cheap_
  + _new_
  + _sure_

+ _necessarily_ (6 unique)
  + _wrong_
  + _easy_
  + _surprising_
  + _representative_
  + _indicative_
  + _true_

+ _before_ (1 unique)
  + _available_

+ _that_ (10 unique)
  + _hard_
  + _complicated_
  + _common_
  + _easy_
  + _great_
  + _good_
  + _uncommon_
  + _simple_
  + _big_
  + _surprising_

+ _remotely_ (3 unique)
  + _interested_
  + _close_
  + _true_

+ _ALL bigrams_ (25 unique)
  + _exactly new_
  + _that simple_
  + _that surprising_
  + _that complicated_
  + _necessarily wrong_
  + _before available_
  + _remotely interested_
  + _that big_
  + _that great_
  + _necessarily surprising_
  + _necessarily easy_
  + _that common_
  + _that hard_
  + _remotely close_
  + _exactly sure_
  + _necessarily true_
  + _exactly clear_
  + _necessarily indicative_
  + _that good_
  + _necessarily representative_
  + _that uncommon_
  + _exactly cheap_
  + _exactly easy_
  + _that easy_
  + _remo


+ _exactly_ (5 unique)
  + _clear_
  + _sure_
  + _new_
  + _easy_
  + _cheap_

+ _necessarily_ (6 unique)
  + _surprising_
  + _true_
  + _representative_
  + _indicative_
  + _easy_
  + _wrong_

+ _before_ (1 unique)
  + _available_

+ _that_ (10 unique)
  + _complicated_
  + _surprising_
  + _easy_
  + _simple_
  + _common_
  + _uncommon_
  + _big_
  + _good_
  + _great_
  + _hard_

+ _remotely_ (3 unique)
  + _close_
  + _interested_
  + _true_

+ _ALL bigrams_ (25 unique)
  + _remotely true_
  + _necessarily indicative_
  + _that hard_
  + _necessarily easy_
  + _exactly easy_
  + _exactly clear_
  + _that big_
  + _that great_
  + _necessarily representative_
  + _exactly cheap_
  + _that surprising_
  + _necessarily surprising_
  + _exactly sure_
  + _that easy_
  + _that common_
  + _that simple_
  + _that complicated_
  + _before available_
  + _necessarily true_
  + _exactly new_
  + _that uncommon_
  + _necessarily wrong_
  + _remotely close_
  + _remotely interested_
  + _that good_

+ _ALL adjectives_ (21 unique)
  + _sure_
  + _new_
  + _indicative_
  + _simple_
  + _common_
  + _interested_
  + _available_
  + _true_
  + _cheap_
  + _close_
  + _complicated_
  + _representative_
  + _easy_
  + _wrong_
  + _clear_
  + _surprising_
  + _uncommon_
  + _big_
  + _good_
  + _great_
  + _hard_


In [21]:
all_top_adv_dfs = [ad['both']
                   for ad in samples_dict.values() if isinstance(ad, dict)]
NEG_bigrams_sample = pd.concat(
    all_top_adv_dfs).sort_values('LRC', ascending=False)
top_NEGbigram_df_path = TOP_AM_DIR.joinpath(
    f'Top{K}_NEG-ADV_top-bigrams.{timestamp_today()}.csv')
print(top_NEGbigram_df_path)
NEG_bigrams_sample.to_csv(
    top_NEGbigram_df_path)
NEG_bigrams_sample

/share/compling/projects/sanpi/results/top_AM/Top5_NEG-ADV_top-bigrams.2024-05-15.csv


,f,exp_f,unexp_f,dP1,LRC,G2,t,MI,...,N,f1,f2,l1,l2,adv,adj,adj_total
key,,,,,,,,,,,,,,,,,
NEG~exactly_sure,8860,347.58,"8,512.42",0.92,8.63,"54,750.58",90.43,1.41,...,86330752,3226213,9301,NEGATED,exactly_sure,exactly,sure,"844,981.00"
NEG~exactly_new,1378,52.99,"1,325.01",0.93,8.54,"8,697.93",35.69,1.42,...,86330752,3226213,1418,NEGATED,exactly_new,exactly,new,"321,311.00"
NEG~that_uncommon,804,30.61,773.39,0.94,8.39,"5,136.91",27.28,1.42,...,86330752,3226213,819,NEGATED,that_uncommon,that,uncommon,"61,767.00"
NEG~necessarily_...,1406,54.41,"1,351.59",0.93,8.37,"8,811.69",36.05,1.41,...,86330752,3226213,1456,NEGATED,necessarily_indi...,necessarily,indicative,"12,760.00"
NEG~exactly_easy,1069,41.11,"1,027.89",0.93,8.37,"6,747.64",31.44,1.42,...,86330752,3226213,1100,NEGATED,exactly_easy,exactly,easy,"771,307.00"
NEG~exactly_clear,1759,68.57,"1,690.43",0.92,8.30,"10,937.16",40.31,1.41,...,86330752,3226213,1835,NEGATED,exactly_clear,exactly,clear,"491,108.00"
NEG~exactly_cheap,693,26.31,666.69,0.95,8.28,"4,443.27",25.33,1.42,...,86330752,3226213,704,NEGATED,exactly_cheap,exactly,cheap,"83,765.00"
NEG~that_surprising,1141,44.36,"1,096.64",0.92,8.14,"7,115.30",32.47,1.41,...,86330752,3226213,1187,NEGATED,that_surprising,that,surprising,"150,067.00"
NEG~that_common,1216,47.39,"1,168.61",0.92,8.12,"7,564.08",33.51,1.41,...,86330752,3226213,1268,NEGATED,that_common,that,common,"556,435.00"


In [22]:
NEG_bigrams_sample.l1.value_counts()

l1
NEGATED    18
NEGMIR      9
Name: count, dtype: Int64